<a href="https://colab.research.google.com/github/Newbie-Nyx/simple-img2img/blob/main/%E8%A9%A6%E4%BD%9CStable_diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install diffusers==0.2.4
!pip install transformers scipy ftfy
!pip install "ipywidgets>=7,<8"

YOUR_TOKEN="\u30C8\u30FC\u30AF\u30F3"#@param {type:"string"}

import torch
from diffusers import StableDiffusionPipeline
from torch import autocast

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", revision="fp16", torch_dtype=torch.float16, use_auth_token=YOUR_TOKEN) 
pipe.to("cuda")



In [ ]:
from google.colab import files        #@title 複数枚出力するコード
from torch.random import seed
from torch import autocast 
from pathlib import Path
import random
import shutil
import os
#-----------------------------------------------------------------------------------------
# NSFWの制限を外す
pipe.safety_checker = lambda images, **kwargs: (images, False)
#-----------------------------------------------------------------------------------------
try:
    shutil.rmtree('/A')               #フォルダAを消す
    os.remove('/AIart.zip')           #zipファイルを消す
except FileNotFoundError:
    pass
#-----------------------------------------------------------------------------------------
path = '/A'                           #フォルダAを作成
os.makedirs(path, exist_ok=True)
#-----------------------------------------------------------------------------------------
i = 1 #@param {type:"slider", min:1, max:50, step:1}
for I in range(i):

  x = random.randint(100000000, 999999999)
  generator = torch.Generator("cuda").manual_seed(x)

  prompt = "style of van gogh"#@param {type:"string"}

  with autocast("cuda"):
    height = 640 #@param {type:"slider", min:512, max:960, step:64}
    width  = 512 #@param {type:"slider", min:512, max:960, step:64}
    Step  = 5 #@param {type:"slider", min:1, max:100, step:1}
    image = pipe(prompt, height, width, num_inference_steps=Step, generator=generator)["sample"][0]

    image.save(f"/A/{x}.png")
#-----------------------------------------------------------------------------------------
if i == 1:
  files.download(f"/A/{x}.png")
else:
  !zip -r /AIart.zip /A                   # フォルダAを zip 圧縮する

  from google.colab import files          # 圧縮したzipファイルをダウンロードする
  files.download("/AIart.zip")

#-----------------------------------------------------------------------------------------
